In [ ]:
pip install coinmetrics-api-client

  Using cached tensorflow-2.19.0-cp311-cp311-win_amd64.whl.metadata (4.1 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached termcolor-3.0.1-py3-none-any.whl.metadata (6.1 kB)
  Using cached wrapt-1.17.2-cp311-cp311-win_amd64.whl.metadata (6.5 kB)
  Using cached grpcio-1.71.0-cp311-cp311-win_amd64.whl.metadata (4.0 kB)
  Using cached tensorboard-2.19.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached keras-3.9.2-py3-none-any.whl.metadata (6.1 kB)
  Using cached numpy-2.1.3-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached h5py-3.13.0-cp311-cp311-win_amd64.whl.metadata (2.5 kB)
  Using cached m

ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'c:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\tensorflow\\compiler\\mlir\\lite\\python\\_pywrap_converter_api.pyd'
Consider using the `--user` option or check the permissions.



  Using cached tensorflow-2.19.0-cp311-cp311-win_amd64.whl.metadata (4.1 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached termcolor-3.0.1-py3-none-any.whl.metadata (6.1 kB)
  Using cached wrapt-1.17.2-cp311-cp311-win_amd64.whl.metadata (6.5 kB)
  Using cached grpcio-1.71.0-cp311-cp311-win_amd64.whl.metadata (4.0 kB)
  Using cached tensorboard-2.19.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached keras-3.9.2-py3-none-any.whl.metadata (6.1 kB)
  Using cached numpy-2.1.3-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached h5py-3.13.0-cp311-cp311-win_amd64.whl.metadata (2.5 kB)
  Using cached m

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scipy 1.10.1 requires numpy<1.27.0,>=1.19.5, but you have numpy 2.1.3 which is incompatible.


In [ ]:
pip install tensorflow

^C
Note: you may need to restart the kernel to use updated packages.


In [6]:
from coinmetrics.api_client import CoinMetricsClient

client = CoinMetricsClient()

print(client)

In [7]:
# 'SplyCntCur' - this is for the supply count of the current coin
# 'CapMrktCurUSD' - this is for the market capitalization of the current coin
# 'PriceUSD' - this is for the price of the current coin
# 'TxTfrValAdjUSD' - this is for the adjusted transaction value in USD
# 'TxTfrValUSD' - this is for the transaction value in USD  
# 'TxTfrCnt' - this is for the transaction count
# 'IssContNtv' - this is for the native issuance count
# 'IssContPctAnn' - this is for the annual percentage of the issuance count
# 
metrics = client.get_asset_metrics(assets='btc', metrics=['PriceUSD','HashRate','TxTfrCnt','AdrActCnt', 'SplyCur'], start_time="2013-01-01", end_time="2023-01-02", frequency='1d')

In [11]:
import pandas as pd

metrics = pd.DataFrame(metrics)

metrics.head()

metrics.to_csv("coin_metricsVar.csv", index=False)

df = metrics.copy()


In [12]:
# Preparing for the data sets for training and testing

# Ensure datetime and sort
df['time'] = pd.to_datetime(df['time'])
df.sort_values('time', inplace=True)

# Create lag features (previous day's metrics)
df['PriceUSD_t-1'] = df['PriceUSD'].shift(1)
df['TxTfrCnt_t-1'] = df['TxTfrCnt'].shift(1)
df['AdrActCnt_t-1'] = df['AdrActCnt'].shift(1)
df['HashRate_t-1'] = df['HashRate'].shift(1)
df['SplyCur_t-1'] = df['SplyCur'].shift(1)

# Create lag features (2 days ago metrics)
df['PriceUSD_t-2'] = df['PriceUSD'].shift(2)
df['TxTfrCnt_t-2'] = df['TxTfrCnt'].shift(2)
df['AdrActCnt_t-2'] = df['AdrActCnt'].shift(2)
df['HashRate_t-2'] = df['HashRate'].shift(2)
df['SplyCur_t-2'] = df['SplyCur'].shift(2)



# Target: future price (e.g., 1-day ahead)
df['PriceUSD_target'] = df['PriceUSD'].shift(-1)

# # Target: future price (e.g., 2-day ahead)
# df['PriceUSD_target'] = df['PriceUSD'].shift(-2)

# # Target: future price (e.g., 3-day ahead)
# df['PriceUSD_target'] = df['PriceUSD'].shift(-3)

# Gives the rolling mean and standard deviation for the last 3 days
# Rolling mean is the average value of the PriceUSD over the last 3 days
# Rolling std is the standard deviation of the PriceUSD over the last 3 days
# df['PriceUSD_roll_mean_3'] = df['PriceUSD'].rolling(window=3).mean()
# df['PriceUSD_roll_std_3'] = df['PriceUSD'].rolling(window=3).std()

df['day_of_week'] = df['time'].dt.dayofweek
df['month'] = df['time'].dt.month

print(df.head())

 # Drop rows with NaNs from shifting
df.dropna(inplace=True)

  asset                      time AdrActCnt          HashRate  \
0   btc 2013-01-01 00:00:00+00:00     38733   23.995203419462   
1   btc 2013-01-02 00:00:00+00:00     40911  22.6621365628253   
2   btc 2013-01-03 00:00:00+00:00     52539   23.995203419462   
3   btc 2013-01-04 00:00:00+00:00     46781  22.6621365628253   
4   btc 2013-01-05 00:00:00+00:00     53959  20.8847140873096   

           PriceUSD            SplyCur TxTfrCnt      PriceUSD_t-1  \
0  13.3313714219755  10618089.91051183    73221              None   
1  13.2806068129749  10621914.90951183    83763  13.3313714219755   
2  13.3840811484512  10625964.90701183    89550  13.2806068129749   
3  13.4517211496201  10629789.90351183    96831  13.3840811484512   
4   13.459406766803  10633314.90101183    86261  13.4517211496201   

  TxTfrCnt_t-1 AdrActCnt_t-1      HashRate_t-1        SplyCur_t-1  \
0         None          None              None               None   
1        73221         38733   23.995203419462  10618089

In [1]:
import numpy as np

def create_lstm_dataset(X, y, time_steps=5):
    Xs, ys = [], []
    for i in range(time_steps, len(X)):
        Xs.append(X[i-time_steps:i])
        ys.append(y[i])
    return np.array(Xs), np.array(ys)

In [2]:
X = df[['HashRate', 'TxTfrCnt', 'AdrActCnt', 'SplyCur', 'PriceUSD_t-1']].values
y = df['PriceUSD_target'].values

# Create sequences
time_steps = 5
X_seq, y_seq = create_lstm_dataset(X, y, time_steps)

# Split manually for LSTM
split = int(0.8 * len(X_seq))
X_train, X_test = X_seq[:split], X_seq[split:]
y_train, y_test = y_seq[:split], y_seq[split:]

NameError: name 'df' is not defined

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model = Sequential()
model.add(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(1))  # Output layer for regression

model.compile(optimizer='adam', loss='mae')
model.summary()

ImportError: Traceback (most recent call last):
  File "c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: A dynamic link library (DLL) initialization routine failed.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

In [1]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


# 2. Model Selection
# Using RandomForestRegressor (you can replace it with any other regressor)
# model = RandomForestRegressor(n_estimators=100, max_depth=20, random_state=42)
model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)

# 3. Model Training
model.fit(X_train, y_train)

# 4. Predictions
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# 5. Evaluation - Calculate MAE
train_mae = mean_absolute_error(y_train, y_train_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)
print(f"Training MAE: {train_mae:.2f}")
print(f"Test MAE: {test_mae:.2f}")

# 6. Visualize Predictions vs Actual Prices
plt.figure(figsize=(10, 5))
plt.plot(y_test.index, y_test, label='Actual Price', color='blue')
plt.plot(y_test.index, y_test_pred, label='Predicted Price', color='red')
plt.title('Actual vs Predicted Price')
plt.xlabel('Date')
plt.ylabel('Price USD')
plt.legend()
plt.grid(True)
plt.show()


NameError: name 'X_train' is not defined

In [24]:
import pandas as pd

df_results = pd.DataFrame({
    'time': df.loc[y_test.index, 'time'],
    'actual': y_test.values,
    'predicted': y_test_pred
})
df_results.to_csv("predictions_simple.csv", index=False)
print("Predictions saved to CSV.")

Predictions saved to CSV.
